In [43]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

In [44]:
department_list =['01','02','03','04','05','06','07','08','09'] + [str(x) for x in xrange(10,96)]
department_list.remove('20')
department_list.extend(['2A','2B','971','972','973','974','976'])

In [45]:
def get_page(address):
    req = requests.get(address)
    soup = BeautifulSoup(req.text,'html.parser')
    return soup

In [46]:
start_time = time.time()

df = pd.DataFrame(columns=['Code Insee','Code Bureau','Address'])
counter = 1

for department_code in department_list:
   
    print counter
    counter +=1
    
    address = "http://www.linternaute.com/ville/gironde/departement-"+ department_code # this link just bring us to the homepage
    homepage = get_page(address)
    #print homepage.title
    link = homepage.find('link')
    vote_page_address = link['href']+'/bureau-vote' # official link to the voting result page
    vote_page = get_page(vote_page_address)
    #print vote_page.title
    commune_list = [t['href'] for t in vote_page.find_all('a', class_="ud", href = re.compile(r'.*bureaux-vote'))]
    
    for commune in commune_list: 
        commune_address = "http://www.linternaute.com" + commune
        #print commune_address
        city = get_page(commune_address)
        #print city.title 
        #print
        s = re.search(ur'2015 [a-z]+ (.*)',city.title.text) #find city name and its postal code
        if not s: 
            s = re.search(ur'à (.*)',city.title.text)
        if not s :
            continue #in case we still can't find the link, or the link is dead, like for Ville in Oise deparment, next to Vignemont
        
        s = re.search(ur'/ville-(.+)/', commune) #find the INSEE code in the html link
        city_code = s.group(1)
        #print city_name
        
        bureau_list = city.find_all(id = re.compile('ville.*')) 
        for bureau in bureau_list: 
            #print bureau
            s = re.search(ur'\d-([\d]+)',bureau['id']) #find bureau code
            if s: 
                bureau_code = s.group(1)
            else: # unique bureau
                bureau_code = 1
            
            bureau_location = ' '.join(bureau.findNext(name='p').strings)
            df.loc[len(df)] = [city_code, bureau_code, bureau_location]
            
print "Run time: ", time.time() - start_time

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
Run time:  14925.255564


In [47]:
df

,Code Insee,Code Bureau,Address
0,01004,1,Gymnase plaine de l'Ain Avenue Léon Blum 01500...
1,01004,2,Gymnase plaine de l'Ain Avenue Léon Blum 01500...
2,01004,3,Château des Echelles 01500 Ambérieu-en-Bugey
3,01004,4,Restaurant scolaire J. Jaurès 01500 Ambérieu-e...
4,01004,5,Restaurant scolaire J. Jaurès 01500 Ambérieu-e...
5,01004,6,Groupe scolaire Jules Ferry Place Jules Ferry ...
6,01004,7,École maternelle de Tiret Rue Jacques Prévert ...
7,01004,8,École maternelle de Tiret Rue Jacques Prévert ...
8,01006,1,Mairie Grande rue 01300 Ambléon
9,01007,1,Salle polyvalente Partie salle des fêtes 01500...


In [48]:
writer = pd.ExcelWriter('bureau_address.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()